Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
- Наименование вакансии.
- Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
- Ссылку на саму вакансию.
- Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [191]:
import requests
import json
import re
import pandas as pd
from bs4 import BeautifulSoup as bs

In [192]:
url = 'https://hh.ru'
route = '/search/vacancy'
params = {
  'text': 'frontend',
  'page': 0
}
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}

In [193]:
vacancies = []

while True:
  response = requests.get(url + route, params=params, headers=headers)
  if response.ok:
    dom = bs(response.text, 'html.parser')

    page_vacancies = dom.select('div.vacancy-serp-item')

    if not page_vacancies:
      break
    
    for page_vacancy in page_vacancies:
      vacancy_data = {
        'salary_from': None,
        'salary_to': None,
        'salary_currency': None
      }

      link_node = page_vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})
      salary_node = page_vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})

      vacancy_data['title'] = link_node.text
      vacancy_data['url'] = link_node['href']
      vacancy_data['from'] = url

      if salary_node:
        salary_list = re.sub('\u202f', '', salary_node.text).split()
        vacancy_data['salary_currency'] = salary_list.pop()

        if salary_list[0] == 'от':
          vacancy_data['salary_from'] = salary_list[1]
          if 'до' in salary_list:
            vacancy_data['salary_to'] = salary_list[3]
        elif salary_list[0] == 'до':
          vacancy_data['salary_to'] = salary_list[1]
        else:
          vacancy_data['salary_from'] = salary_list[0]
          if '–' in salary_list:
            vacancy_data['salary_from'] = salary_list[0]
            vacancy_data['salary_to'] = salary_list[2]
          else:
            vacancy_data['salary_to'] = salary_list[0]

      vacancies.append(vacancy_data)

    print(f"Обработано вакансий на {params['page'] + 1} странице: {len(page_vacancies)}")
    params['page'] += 1
  else:
    break

Обработано вакансий на 1 странице: 20
Обработано вакансий на 2 странице: 20
Обработано вакансий на 3 странице: 20
Обработано вакансий на 4 странице: 20
Обработано вакансий на 5 странице: 20
Обработано вакансий на 6 странице: 20
Обработано вакансий на 7 странице: 20
Обработано вакансий на 8 странице: 20
Обработано вакансий на 9 странице: 20
Обработано вакансий на 10 странице: 20
Обработано вакансий на 11 странице: 20
Обработано вакансий на 12 странице: 20
Обработано вакансий на 13 странице: 20
Обработано вакансий на 14 странице: 20
Обработано вакансий на 15 странице: 20
Обработано вакансий на 16 странице: 20
Обработано вакансий на 17 странице: 20
Обработано вакансий на 18 странице: 20
Обработано вакансий на 19 странице: 20
Обработано вакансий на 20 странице: 20
Обработано вакансий на 21 странице: 20
Обработано вакансий на 22 странице: 20
Обработано вакансий на 23 странице: 20
Обработано вакансий на 24 странице: 20
Обработано вакансий на 25 странице: 20
Обработано вакансий на 26 странице

In [198]:
data = pd.DataFrame(vacancies)
data.sample(10)

,salary_from,salary_to,salary_currency,title,url,from
24,50000,100000,руб.,Frontend-разработчик,https://hh.ru/vacancy/48623430?from=vacancy_se...,https://hh.ru
38,None,None,None,Junior Frontend Developer,https://hh.ru/vacancy/49183771?from=vacancy_se...,https://hh.ru
752,None,None,None,Frontend-разработчик,https://hh.ru/vacancy/47355508?from=vacancy_se...,https://hh.ru
394,None,4000,USD,Frontend React Developer (разработка CRM-платф...,https://hh.ru/vacancy/46941963?from=vacancy_se...,https://hh.ru
689,100000,150000,руб.,Middle Front-end Developer (Vue.js),https://hh.ru/vacancy/47201449?from=vacancy_se...,https://hh.ru
51,80000,150000,руб.,Frontend разработчик,https://hh.ru/vacancy/48983613?from=vacancy_se...,https://hh.ru
30,150000,None,руб.,Middle Frontend developer,https://hh.ru/vacancy/48807548?from=vacancy_se...,https://hh.ru
443,None,None,None,Frontend-разработчик (удалённо),https://hh.ru/vacancy/42701476?from=vacancy_se...,https://hh.ru
150,None,200000,руб.,Frontend-разработчик,https://hh.ru/vacancy/48920504?from=vacancy_se...,https://hh.ru
332,250000,None,руб.,"Senior Frontend Developer (Angular, удаленно)",https://hh.ru/vacancy/46096347?from=vacancy_se...,https://hh.ru


In [195]:
with open('hh_vacancies.json', 'w') as f:
    json.dump(vacancies, f)